In [41]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import keras
from PIL import Image

print("Setup complete")

Setup complete


In [42]:
fully_covered = os.listdir("data/fully_covered/")
not_covered = os.listdir("data/not_covered/")
not_face = os.listdir("data/not_face/")
partially_covered = os.listdir("data/partially_covered/")


In [43]:
img_w,img_h = 64, 64

LABELS = ["Fully Covered", "Not Covered", "Not a Face", "Partially Covered"]

X = []
y = []

In [44]:
for i in fully_covered:
    try:            
        img = Image.open("data/fully_covered/" + i).convert('RGB') # getting each image an converting to RGB
        img.resize((img_w, img_h)) # scaling the size down
        X.append(np.asarray(img)) # adding the array value of each image to our data set
        y.append(0) # adding the label
    except:
        pass



In [45]:
for i in not_covered:
    try:
        img = Image.open("data/not_covered/" + i ).convert('RGB')
        img.resize((img_w,img_h))
        X.append(np.asarray(img))
        y.append(1)
    except:
        pass

In [46]:
for i in not_face:
    try:
        img = Image.open("data/not_face/" + i).convert('RGB')
        img.resize((img_w,img_h))
        X.append(np.asarray(img))
        y.append(2)
    except:
        pass

In [47]:
for i in partially_covered:
    try:
        img = Image.open("data/partially_covered/" + i).convert('RGB')
        img.resize((img_w,img_h))
        X.append(np.asarray(img))
        y.append(3)
    except:
        pass

In [50]:
X = np.asarray(X)
y = np.asarray(y)

X = X.astype('float32')
X /= 255 # convert this to a number between 0-1

In [57]:
num_classes = 4
labels = keras.utils.to_categorical(y=y, num_classes=num_classes)

In [ ]:
keras.